In [2]:
import jax
import jax.numpy as jnp
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
# load the CSV file into a Pandas DataFrame
df = pd.read_csv("data.csv")

In [13]:
df

,formula,elements,nsites,element_symbols,coordinates,energy,spacegroup,spacegroup_num
0,Ag11Hg9,"Ag, Hg",20,"['Ag', 'Ag', 'Ag', 'Ag', 'Ag', 'Ag', 'Ag', 'Ag...",[[ 1.609567 -0.92928553 33.90805742]\n [ 1.6...,-677.652749,P3m1,156
1,Ag2Br3,"Ag, Br",10,"['Ag', 'Ag', 'Ag', 'Ag', 'Br', 'Br', 'Br', 'Br...",[[-3.46990705e+00 -2.00392334e+00 1.22402099e...,-168.691087,R-3c,167
2,Ag2Cl3,"Ag, Cl",10,"['Ag', 'Ag', 'Ag', 'Ag', 'Cl', 'Cl', 'Cl', 'Cl...",[[-3.30941698e+00 -1.91047971e+00 1.71721380e...,-127.332050,R-3c,167
3,Ag2F,"Ag, F",3,"['Ag', 'Ag', 'F']",[[-1.83755070e-06 1.72922338e+00 3.97018472e...,-48.352742,P-3m1,164
4,Ag2F3,"Ag, F",10,"['Ag', 'Ag', 'Ag', 'Ag', 'F', 'F', 'F', 'F', '...",[[-2.87395102e+00 -1.65949299e+00 2.26233906e...,-116.183864,R-3c,167
...,...,...,...,...,...,...,...,...
10623,ZrZn16,"Zr, Zn",34,"['Zr', 'Zr', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn...",[[-2.17422601 6.06378581 8.71633518]\n [ 6.4...,-333.573781,Cmcm,63
10624,ZrZn2,"Zr, Zn",6,"['Zr', 'Zr', 'Zn', 'Zn', 'Zn', 'Zn']",[[ 5.25615317e+00 3.71666061e+00 9.10392570e...,-83.354040,Fd-3m,227
10625,ZrZn22,"Zr, Zn",46,"['Zr', 'Zr', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn...",[[ 4.26511414e+00 3.01589110e+00 7.38739309e...,-440.455327,Fd-3m,227
10626,ZrZn3,"Zr, Zn",8,"['Zr', 'Zr', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn', 'Zn']",[[ 2.24703936e-06 3.37600716e+00 3.24801562e...,-101.493900,P6_3/mmc,194


In [4]:
# create a one-hot encoding of the elements in the "elements" column
elements = list(set([el for row in df['elements'].str.split(',') for el in row]))
for element in elements:
    df[element] = df["elements"].apply(lambda x: element in x).astype(int)

/tmp/ipykernel_8207/3177165320.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[element] = df["elements"].apply(lambda x: element in x).astype(int)
/tmp/ipykernel_8207/3177165320.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[element] = df["elements"].apply(lambda x: element in x).astype(int)
/tmp/ipykernel_8207/3177165320.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

In [5]:
# create the feature matrix X and the target variable y
X = df[elements].values
y = df["spacegroup_num"].values

In [6]:
X.shape

(10628, 156)

In [7]:
print(X[:5])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
  0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [8]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# Train the random forest classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
# Make predictions on the test set
y_pred = rfc.predict(X_test)

In [11]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.0973659454374412
